In [3]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from fuzzywuzzy import fuzz

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


c:\Users\jsbreite\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning:

LightFM was compiled without OpenMP support. Only a single thread will be used.



In [1]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Combined_Data_All.gzip'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [4]:
# Get Data

data = pd.read_parquet(new_Combined)
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64'}
data= data.astype(Convert_dic_data)
del new_Combined

# Get Movie_Titles
movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Target'])
                           # Speicher alle Daten erst in eine Reihe, danach trennt er diese
movie_titles[['Movie_Id', 'Year', 'Name']] = movie_titles['Target'].str.split(pat=",",n=2, expand=True)   
movie_titles= movie_titles.drop(['Target', 'Year'], axis= 1)  
Convert_dic= {'Movie_Id': 'int64', 'Name': 'str'}
movie_titles = movie_titles.astype(Convert_dic)
#movie_titles.astype({'Movie_Id': 'int64', 'Name': 'str'}).dtypes
#movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
#movie_titles = movie_titles.reset_index(drop=True)
display(movie_titles)

del Convert_dic

,Movie_Id,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW
...,...,...
17765,17766,Where the Wild Things Are and Other Maurice Se...
17766,17767,Fidel Castro: American Experience
17767,17768,Epoch
17768,17769,The Company


In [5]:
df = data
# Filter sparse movies
min_movie_ratings = 1000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]

print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))
del filter_movies, filter_users, min_movie_ratings, min_user_ratings, data, df

Shape User-Ratings unfiltered:	(100480507, 3)
Shape User-Ratings filtered:	(75010415, 3)


In [7]:
df_filterd_name = pd.merge(df_filterd,movie_titles.drop_duplicates(),on='Movie_Id', how='inner')
print(df_filterd_name.head(2))
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64', 'Name': 'str'}
df_filterd_name = df_filterd_name.astype(Convert_dic_data)
del Convert_dic_data

   Cust_Id  Rating  Movie_Id       Name
0   712664     5.0         3  Character
1  1331154     4.0         3  Character


In [18]:
#print((~df_filterd_name.duplicated()).sum())
print(df_filterd.shape)
print(df_filterd_name.shape)


(75010415, 3)
(75010415, 4)


In [10]:
movie_user_mat_ID = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

In [6]:
# Get a list of all Variables 
df_movie_id_list = df_filterd_name.Name.unique().tolist()
df_movie_id_list = pd.DataFrame(df_movie_id_list)
print(df_movie_id_list.head(4))

# Get a List of all user
movie_index_list = list(movie_user_mat.index.values)

                            0
0                   Character
1    The Rise and Fall of ECW
2                        Sick
3  What the #$*! Do We Know!?


In [12]:
user_movie_table_matrix = csr_matrix(movie_user_mat_ID.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto',n_jobs=-1)
model_knn.fit(user_movie_table_matrix)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [31]:
query_index = np.random.choice(movie_user_mat_ID.shape[0])

movie_id = 17627

movie_id_mat = movie_user_mat_ID.loc[[movie_id]].values.reshape(1,-1)
distances, indices = model_knn.kneighbors(movie_id_mat, n_neighbors = 7000)

In [25]:
# Iteration aufbauen
distances, indices = model_knn.kneighbors(movie_user_mat_ID.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 5000)

In [32]:
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(movie_user_mat_ID.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='Movie_Id')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(movie_user_mat_ID.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))

Recommendations for 11679:

0: 13176, with distance of 0.956211268901825
1: 206, with distance of 0.9560534358024597
2: 6919, with distance of 0.9560270309448242
3: 5072, with distance of 0.9560205936431885
4: 16730, with distance of 0.9560143947601318
5: 2684, with distance of 0.9559513926506042
6: 12344, with distance of 0.9559404850006104
7: 1114, with distance of 0.9559196829795837
8: 8961, with distance of 0.9558587670326233
9: 13366, with distance of 0.9555720686912537
10: 17025, with distance of 0.9555570483207703
11: 17059, with distance of 0.9555166959762573
12: 410, with distance of 0.9554939866065979
13: 8297, with distance of 0.9554590582847595
14: 6584, with distance of 0.9554476737976074
15: 8835, with distance of 0.9554370045661926
16: 56, with distance of 0.9554134011268616
17: 4869, with distance of 0.9553861021995544
18: 707, with distance of 0.9553760290145874
19: 13419, with distance of 0.9553440809249878
20: 9873, with distance of 0.9552867412567139
21: 9727, with 